# PyCity Schools Analysis

* As a whole, schools with higher budgets, did not yield better test results. By contrast, schools with higher spending per student actually (\$645-675) underperformed compared to schools with smaller budgets (<\$585 per student).

* As a whole, smaller and medium sized schools dramatically out-performed large sized schools on passing math performances (89-91% passing vs 67%).

* As a whole, charter schools out-performed the public district schools across all metrics. However, more analysis will be required to glean if the effect is due to school practices or the fact that charter schools tend to serve smaller student populations per school. 
---

### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import os
import csv
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
school_data_to_load = os.path.join("Resources", "schools_complete.csv")
student_data_to_load = os.path.join("Resources", "students_complete.csv")

# Read School and Student Data File and store into Pandas Data Frames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

#Let's format the field to show
school_data_complete["budget"] = school_data_complete["budget"].map("${:,.2f}".format)
school_data_complete.head()


,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,"$1,910,635.00"
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,"$1,910,635.00"
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,"$1,910,635.00"
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,"$1,910,635.00"
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,"$1,910,635.00"


## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [2]:
#Format the field budget to numeric
school_data_complete["budget"] = school_data_complete["budget"].replace({'\$':'',',':''}, regex = True)
school_data_complete["budget"] = school_data_complete["budget"].astype(float)
#Calculate the total number of schools
school_count = len(school_data["School ID"].unique())
#Calculate the total number of students
student_count = len(student_data["Student ID"].unique())
#Calculate the total budget
total_budget = school_data_complete["budget"].sum()
#Calculate the average math score
av_math = round(school_data_complete["math_score"].mean(),2)
#Calculate the average reading score
av_reading = round(school_data_complete["reading_score"].mean(),2)
#Calculate the overall passing rate
ov_passing = round((av_math + av_reading)/2,2)
#Calculate the percentage of students with a passing math score (70 or greater)
math_greater = sum(school_data_complete["math_score"] > 69)
perc_math = round((math_greater/student_count)*100,2)
#Calculate the percentage of students with a passing reading score (70 or greater)
reading_greater = sum(school_data_complete["reading_score"]>69)
perc_reading = round((reading_greater/student_count)*100,2)

# Place all of the data found into a summary DataFrame
summary_table = pd.DataFrame({"Total Schools": [school_count],
                              "Total Students": [student_count],
                              "Total Budget": [total_budget],
                              "Average Math Score": [av_math],
                              "Average Reading Score": [av_reading],
                              "% Passing Math": [perc_math],
                              "% Passing Reading": [perc_reading],
                              "Overall Passing Rate": [ov_passing]})
summary_table

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
0,15,39170,8.293233e+10,78.99,81.88,74.98,85.81,80.44


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)
  
* Create a dataframe to hold the above results

In [3]:
#School Summary

schoolName = school_data["school_name"]
schoolType = school_data["type"]
total_students = school_data["size"]
total_budget = school_data["budget"]
student_budget = total_budget / total_students


av_math_score = round(school_data_complete.groupby(["school_name"])["math_score"].mean(),2)
av_reading_score = round(school_data_complete.groupby(["school_name"])["reading_score"].mean(),2)
 
math_passing = school_data_complete[student_data.math_score>=70].groupby('school_name')['math_score'].count()/school_data_complete.groupby(["school_name"])["Student ID"].count()

perc_math = round(math_passing *100,2)

reading_passing = school_data_complete[student_data.reading_score>=70].groupby('school_name')['reading_score'].count()/school_data_complete.groupby(["school_name"])["Student ID"].count()
perc_reading = round(reading_passing * 100,2)

ovpassing = round((perc_math + perc_reading)/2,2)

#Place all of the data found into a summary DataFrame
#School Summary
school_summary = pd.DataFrame({"school_name": schoolName,
                               "School Type": schoolType,
                               "Total Students":total_students,
                               "Total School Budget":total_budget,
                               "Per Student Budget":student_budget 
                              })
# Students Summary
students_summary = pd.DataFrame({"Average Math Score":av_math_score,
                                 "Average Reading Score":av_reading_score,
                                 "% Passing Math":perc_math,
                                 "% Passing Reading":perc_reading,
                                 "Overall Passing Rate":ovpassing})

#Merge Schools and Students summary
merged_summary = pd.merge(school_summary, students_summary, on="school_name",how='inner')
merged_summary = merged_summary.rename(columns={"school_name":"School Name"})
#format currency data to show
merged_summary["Total School Budget"] = merged_summary["Total School Budget"].map("${:,.2f}".format)
merged_summary["Per Student Budget"] = merged_summary["Per Student Budget"].map("${:,.2f}".format)
merged_summary



,School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
0,Huang High School,District,2917,"$1,910,635.00",$655.00,76.63,81.18,65.68,81.32,73.50
1,Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.71,81.16,65.99,80.74,73.36
2,Shelton High School,Charter,1761,"$1,056,600.00",$600.00,83.36,83.73,93.87,95.85,94.86
3,Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.29,80.93,66.75,80.86,73.81
4,Griffin High School,Charter,1468,"$917,500.00",$625.00,83.35,83.82,93.39,97.14,95.26
5,Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.27,83.99,93.87,96.54,95.21
6,Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.06,83.98,94.13,97.04,95.58
7,Bailey High School,District,4976,"$3,124,928.00",$628.00,77.05,81.03,66.68,81.93,74.31
8,Holden High School,Charter,427,"$248,087.00",$581.00,83.80,83.81,92.51,96.25,94.38
9,Pena High School,Charter,962,"$585,858.00",$609.00,83.84,84.04,94.59,95.95,95.27


## Top Performing Schools (By Passing Rate)

* Sort and display the top five schools in overall passing rate

In [4]:
topPerformer_df = merged_summary.sort_values("Overall Passing Rate", ascending=False)
indexTopPerformer_df = topPerformer_df.reset_index(drop=True)
indexTopPerformer_df.head(5)


,School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
0,Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.06,83.98,94.13,97.04,95.58
1,Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.42,83.85,93.27,97.31,95.29
2,Pena High School,Charter,962,"$585,858.00",$609.00,83.84,84.04,94.59,95.95,95.27
3,Griffin High School,Charter,1468,"$917,500.00",$625.00,83.35,83.82,93.39,97.14,95.26
4,Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.27,83.99,93.87,96.54,95.21


## Bottom Performing Schools (By Passing Rate)

* Sort and display the five worst-performing schools

In [5]:
worstPerformer_df = merged_summary.sort_values("Overall Passing Rate", ascending=True)
indexworstPerformer_df = worstPerformer_df.reset_index(drop=True)
indexworstPerformer_df.head(5)

,School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
0,Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.84,80.74,66.37,80.22,73.30
1,Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.71,81.16,65.99,80.74,73.36
2,Huang High School,District,2917,"$1,910,635.00",$655.00,76.63,81.18,65.68,81.32,73.50
3,Johnson High School,District,4761,"$3,094,650.00",$650.00,77.07,80.97,66.06,81.22,73.64
4,Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.29,80.93,66.75,80.86,73.81


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [6]:
ninth_math = school_data_complete[(school_data_complete["grade"] == "9th")]
tenth_math = school_data_complete[(school_data_complete["grade"] == "10th")]
eleventh_math = school_data_complete[(school_data_complete["grade"] == "11th")]
twelfth_math = school_data_complete[(school_data_complete["grade"] == "12th")]

ninthScoreMath = round(ninth_math.groupby("school_name")["math_score"].mean(),2)
tenthScoreMath = round(tenth_math.groupby("school_name")["math_score"].mean(),2)
eleventhScoreMath = round(eleventh_math.groupby("school_name")["math_score"].mean(),2)
twelfthScoreMath = round(twelfth_math.groupby("school_name")["math_score"].mean(),2)

mathScores=pd.DataFrame({
    "9th":ninthScoreMath,
    "10th":tenthScoreMath,
    "11th":eleventhScoreMath,
    "12th":twelfthScoreMath
})
mathScores

,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.08,77.00,77.52,76.49
Cabrera High School,83.09,83.15,82.77,83.28
Figueroa High School,76.40,76.54,76.88,77.15
Ford High School,77.36,77.67,76.92,76.18
Griffin High School,82.04,84.23,83.84,83.36
Hernandez High School,77.44,77.34,77.14,77.19
Holden High School,83.79,83.43,85.00,82.86
Huang High School,77.03,75.91,76.45,77.23
Johnson High School,77.19,76.69,77.49,76.86


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [7]:
ninth_reading = school_data_complete[(school_data_complete["grade"] == "9th")]
tenth_reading = school_data_complete[(school_data_complete["grade"] == "10th")]
eleventh_reading = school_data_complete[(school_data_complete["grade"] == "11th")]
twelfth_reading = school_data_complete[(school_data_complete["grade"] == "12th")]

ninthScoreRead = round(ninth_reading.groupby("school_name")["reading_score"].mean(),2)
tenthScoreRead = round(tenth_reading.groupby("school_name")["reading_score"].mean(),2)
eleventhScoreRead = round(eleventh_reading.groupby("school_name")["reading_score"].mean(),2)
twelfthScoreRead = round(twelfth_reading.groupby("school_name")["reading_score"].mean(),2)

readingScores=pd.DataFrame({
    "9th":ninthScoreRead,
    "10th":tenthScoreRead,
    "11th":eleventhScoreRead,
    "12th":twelfthScoreRead
})
readingScores

,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.30,80.91,80.95,80.91
Cabrera High School,83.68,84.25,83.79,84.29
Figueroa High School,81.20,81.41,80.64,81.38
Ford High School,80.63,81.26,80.40,80.66
Griffin High School,83.37,83.71,84.29,84.01
Hernandez High School,80.87,80.66,81.40,80.86
Holden High School,83.68,83.32,83.82,84.70
Huang High School,81.29,81.51,81.42,80.31
Johnson High School,81.26,80.77,80.62,81.23


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [8]:
# Sample bins. Feel free to create your own bins.
spending_bins = [0, 585, 615, 645, 675]
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]
# Place the data series into a new column inside of the DataFrame student_budget
merged_summary["Spending Range"] = pd.cut(student_budget, spending_bins, labels=group_names)
# merged_summary.head()
spendingMath = merged_summary.groupby("Spending Range").mean()["Average Math Score"]
spendingReading = merged_summary.groupby("Spending Range").mean()["Average Reading Score"]
spendingMathPass = merged_summary.groupby("Spending Range").mean()["% Passing Math"]
spendingReadPass = merged_summary.groupby("Spending Range").mean()["% Passing Reading"]
spendingOverallPass = (spendingMath + spendingReading)/ 2

school_spending = pd.DataFrame({
    "Average Math Score":round(spendingMath,2),
    "Average Reading Score":round(spendingReading,2),
    "Passing Math":round(spendingMathPass,2),
    "Passing Reading":round(spendingReadPass,2),
    "Passing Overall":round(spendingOverallPass,2)
})
school_spending


,Average Math Score,Average Reading Score,Passing Math,Passing Reading,Passing Overall
Spending Range,,,,,
<$585,83.45,83.94,93.46,96.61,83.69
$585-615,83.60,83.88,94.23,95.90,83.74
$615-645,79.08,81.89,75.67,86.11,80.49
$645-675,77.00,81.03,66.16,81.13,79.01


## Scores by School Size

* Perform the same operations as above, based on school size.

In [9]:
# Sample bins. Feel free to create your own bins.
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

merged_summary["Total of Students"]=pd.cut(total_students,size_bins,labels=group_names)

schoolSizeMath = merged_summary.groupby("Total of Students").mean()["Average Math Score"]
schoolSizeRead = merged_summary.groupby("Total of Students").mean()["Average Reading Score"]
schoolSizeMathPass = merged_summary.groupby("Total of Students").mean()["% Passing Math"]
schoolSizeReadPass = merged_summary.groupby("Total of Students").mean()["% Passing Reading"]
schoolSizeOvPass = (schoolSizeMathPass + schoolSizeReadPass)/2

school_size = pd.DataFrame({
    "Average Math Score":round(schoolSizeMath,2),
    "Average Reading Score":round(schoolSizeRead,2),
    "Passing Math":round(schoolSizeMathPass,2),
    "Passing Reading":round(schoolSizeReadPass,2),
    "Passing Overall":round(schoolSizeOvPass,2)
})
school_size

,Average Math Score,Average Reading Score,Passing Math,Passing Reading,Passing Overall
Total of Students,,,,,
Small (<1000),83.82,83.93,93.55,96.10,94.82
Medium (1000-2000),83.37,83.87,93.60,96.79,95.19
Large (2000-5000),77.74,81.34,69.96,82.77,76.37


## Scores by School Type

* Perform the same operations as above, based on school type.

In [10]:
schoolTypeMath = merged_summary.groupby("School Type").mean()["Average Math Score"]
schoolTypeRead = merged_summary.groupby("School Type").mean()["Average Reading Score"]
schoolTypeMathPass = merged_summary.groupby("School Type").mean()["% Passing Math"]
schoolTypeReadPass = merged_summary.groupby("School Type").mean()["% Passing Reading"]
schoolTypeOvPass = (schoolTypeMathPass + schoolTypeReadPass)/2

school_type = pd.DataFrame({
    "Average Math Score":round(schoolTypeMath,2),
    "Average Reading Score":round(schoolTypeRead,2),
    "Passing Math":round(schoolTypeMathPass,2),
    "Passing Reading":round(schoolTypeReadPass,2),
    "Passing Overall":round(schoolTypeOvPass,2)
})
school_type

,Average Math Score,Average Reading Score,Passing Math,Passing Reading,Passing Overall
School Type,,,,,
Charter,83.47,83.90,93.62,96.59,95.10
District,76.96,80.97,66.55,80.80,73.67
